In [1]:
import xarray as xr
import xesmf as xe
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import dask
import cftime
import random
import glob
from scipy.signal import welch
import matplotlib.gridspec as gs

In [2]:
from dask.distributed import Client, LocalCluster
client = Client()
client

2024-01-16 15:25:55,688 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/106275402.gadi-pbs/dask-worker-space/worker-qh4hl2o7', purging
2024-01-16 15:25:55,688 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/106275402.gadi-pbs/dask-worker-space/worker-xvfbvujk', purging
2024-01-16 15:25:55,689 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/106275402.gadi-pbs/dask-worker-space/worker-zf9v2gah', purging
2024-01-16 15:25:55,689 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/106275402.gadi-pbs/dask-worker-space/worker-8559saj6', purging
2024-01-16 15:25:55,689 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/106275402.gadi-pbs/dask-worker-space/worker-smy3xtgq', purging
2024-01-16 15:25:55,689 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/106275402.gadi-pbs/dask-worker-space/worker-blc_020z', purging
2024-01-16 15:25

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33965,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:44145,Total threads: 1
Dashboard: /proxy/35597/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:32791,


In [3]:
def regrid_ACCESS(ds, var):
    #rename coords
    ds = ds.rename({"geolon_t": "longitude", "geolat_t": "latitude"})
    #create the output grid
    ds_out = xe.util.grid_global(1, 1)
    ds_out = ds_out.drop({'lon_b', 'lat_b'})
    #rename grid coords
    ds_out = ds_out.rename({'lon': 'longitude', 'lat': 'latitude'})
    #drop the coords on input ds
    ds_in_1deg = ds.drop({'xt_ocean', 'yt_ocean'})
    #create regridder
    regridder_1deg = xe.Regridder(ds_in_1deg, ds_out, 'bilinear', periodic=True)
    #now regrid
    regridded = regridder_1deg(ds[var].chunk({'xt_ocean': -1, 'yt_ocean': -1}))
    regridded = regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
    regridded = regridded.rename({'x': 'longitude', 'y': 'latitude'})
    return regridded

In [4]:
ctrl_sst = xr.open_dataset("/g/data/e14/sm2435/Exps_ACCESS_initial/ctrl_sst.nc",  use_cftime=True, chunks= {"time":240})
ctrl = regrid_ACCESS(ctrl_sst, 'sst')
ctrl_sst = ctrl_sst.sst.compute()

In [5]:
de_had = xr.open_dataset("/g/data/e14/sm2435/Pacemaker/hadisst_clim_1x1.nc").__xarray_dataarray_variable__

In [6]:
de_had

<xarray.DataArray '__xarray_dataarray_variable__' (month: 12, latitude: 180,
                                                   longitude: 360)>
[777600 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [7]:
ctrl_mn = ctrl_sst.mean("time") 

In [8]:
ctrl_mn = ctrl_mn.compute()

In [ ]:
(ctrl_mn - de_had.mean("month")).plot()

Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2024-01-16 15:30:00,010 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-01-16 15:30:00,010 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2024-01-16 15:30:00,011 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-01-16 15:30:00,010 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/pyth

In [ ]:
(ctrl_sst.groupby("time.month").mean("time") - de_had).plot(col = "month", col_wrap = 4)

In [ ]:
(ctrl_sst.mean("time") - de_had.mean("month")).plot()